### Setup

In [32]:
from openai import OpenAI
from dotenv import load_dotenv
import json
from pprint import pprint
from IPython.display import display, Markdown

load_dotenv()

client = OpenAI()

### Unstructured to Structured

In [19]:
system_prompt = "You are an assistant that returns only JSON objects with the requested information."

def complete(user_prompt):
    completion = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
        max_tokens=400,
    )
    return completion

def get_response(completion):
    return completion.choices[0].message.content

In [5]:
prompt = "Please return a JSON object about Albert Einstein. Include full name, birth date, date of death (if applicable), and the list of accomplishments."
completion = complete(prompt)
completion

ChatCompletion(id='chatcmpl-9jHxjQMboj2MMGNZ9FRLPc3lxzQFY', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n    "full_name": "Albert Einstein",\n    "birth_date": "March 14, 1879",\n    "date_of_death": "April 18, 1955",\n    "accomplishments": [\n        "Theory of General Relativity",\n        "Mass-energy equivalence formula E=mc^2",\n        "Photoelectric effect explanation leading to Nobel Prize in Physics",\n        "Contributions to quantum mechanics"\n    ]\n}', role='assistant', function_call=None, tool_calls=None))], created=1720580835, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=90, prompt_tokens=54, total_tokens=144))

In [7]:
print(get_response(completion))

{
    "full_name": "Albert Einstein",
    "birth_date": "March 14, 1879",
    "date_of_death": "April 18, 1955",
    "accomplishments": [
        "Theory of General Relativity",
        "Mass-energy equivalence formula E=mc^2",
        "Photoelectric effect explanation leading to Nobel Prize in Physics",
        "Contributions to quantum mechanics"
    ]
}


In [9]:
ae = json.loads(get_response(completion))
ae

{'full_name': 'Albert Einstein',
 'birth_date': 'March 14, 1879',
 'date_of_death': 'April 18, 1955',
 'accomplishments': ['Theory of General Relativity',
  'Mass-energy equivalence formula E=mc^2',
  'Photoelectric effect explanation leading to Nobel Prize in Physics',
  'Contributions to quantum mechanics']}

In [10]:
# Turn it into a function that will tell us about anyone!
def get_person(name):
    prompt = f"Please return a JSON object about {name}. Include full name, birth date, date of death (if applicable), and the list of accomplishments."
    completion = complete(prompt)
    data = json.loads(get_response(completion))
    return data

In [13]:
pprint(get_person("Steve Jobs"))

{'accomplishments': ['Co-founder of Apple Inc.',
                     'Pioneer of the personal computer revolution',
                     'Creator of the iPod, iPhone, and iPad',
                     'Visionary leader in the tech industry'],
 'birth_date': 'February 24, 1955',
 'date_of_death': 'October 5, 2011',
 'full_name': 'Steve Jobs'}


In [14]:
pprint(get_person("Bill Gates"))

{'accomplishments': ['Co-founder of Microsoft Corporation',
                     'One of the wealthiest individuals in the world',
                     'Philanthropist through the Bill & Melinda Gates '
                     'Foundation'],
 'birth_date': 'October 28, 1955',
 'date_of_death': None,
 'full_name': 'Bill Gates'}


In [15]:
report = '''
CUPERTINO, CALIFORNIA Apple today announced financial results for its fiscal 2024 second quarter ended March 30, 2024. The Company posted quarterly revenue of $90.8 billion, down 4 percent year over year, and quarterly earnings per diluted share of $1.53.
“Today Apple is reporting revenue of $90.8 billion for the March quarter, including an all-time revenue record in Services,” said Tim Cook, Apple’s CEO. “During the quarter, we were thrilled to launch Apple Vision Pro and to show the world the potential that spatial computing unlocks. We’re also looking forward to an exciting product announcement next week and an incredible Worldwide Developers Conference next month. As always, we are focused on providing the very best products and services for our customers, and doing so while living up to the core values that drive us.”
“Thanks to very high levels of customer satisfaction and loyalty, our active installed base of devices has reached a new all-time high across all products and all geographic segments, and our business performance drove a new EPS record for the March quarter,” said Luca Maestri, Apple’s CFO. “Given our confidence in Apple’s future and the value we see in our stock, our Board has authorized an additional $110 billion for share repurchases. We are also raising our quarterly dividend for the twelfth year in a row.”
Apple’s board of directors has declared a cash dividend of $0.25 per share of the Company’s common stock, an increase of 4 percent. The dividend is payable on May 16, 2024 to shareholders of record as of the close of business on May 13, 2024. The board of directors has also authorized an additional program to repurchase up to $110 billion of the Company’s common stock.
Apple will provide live streaming of its Q2 2024 financial results conference call beginning at 2:00 p.m. PT on May 2, 2024 at apple.com/investor/earnings-call. The webcast will be available for replay for approximately two weeks thereafter.
'''

In [16]:
prompt = f"""
Please pull all the important information from the below quarterly earnings report, enclosed in triple backticks. Return the result as a JSON. \n\n
```
{report}
```
"""

In [22]:
completion = complete(prompt)
data = json.loads(get_response(completion))
data

{'company': 'Apple',
 'quarter': 'Q2 2024',
 'revenue': '$90.8 billion',
 'revenue_change': '-4%',
 'earnings_per_share': '$1.53',
 'CEO': 'Tim Cook',
 'CFO': 'Luca Maestri',
 'dividend_per_share': '$0.25',
 'dividend_increase': '4%',
 'dividend_payable_date': 'May 16, 2024',
 'dividend_record_date': 'May 13, 2024',
 'share_repurchase_program': '$110 billion',
 'earnings_call_date': 'May 2, 2024',
 'earnings_call_time': '2:00 p.m. PT',
 'earnings_call_webcast': 'apple.com/investor/earnings-call'}

In [23]:
prompt = f"""
Please pull all the important information from the below quarterly earnings report, enclosed in triple backticks. Return the result as a JSON.

Include the keys company, ceo, cfo, quarter, revenue, revenue_change, earnings_per_share, dividend_per_share, dividend_increase, dividend_payable_date, \
dividend_record_date, share_repurchase_program, earnings_call_date, earnings_call_time and earnings_call_webcast.
\n\n
```
{report}
```
"""

In [24]:
completion = complete(prompt)
data = json.loads(get_response(completion))
data

{'company': 'Apple',
 'ceo': 'Tim Cook',
 'cfo': 'Luca Maestri',
 'quarter': 'Q2 2024',
 'revenue': '$90.8 billion',
 'revenue_change': '-4%',
 'earnings_per_share': '$1.53',
 'dividend_per_share': '$0.25',
 'dividend_increase': '4%',
 'dividend_payable_date': 'May 16, 2024',
 'dividend_record_date': 'May 13, 2024',
 'share_repurchase_program': '$110 billion',
 'earnings_call_date': 'May 2, 2024',
 'earnings_call_time': '2:00 p.m. PT',
 'earnings_call_webcast': 'apple.com/investor/earnings-call'}

### Structured to Unstructured

In [33]:
system_prompt = "You are an assistant that writes concise, detailed, and factual quarterly earnings reports given structured data."

def complete(user_prompt):
    completion = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
        #max_tokens=400,
    )
    return completion

def print_response(completion):
    content = completion.choices[0].message.content
    display(Markdown(content))

In [27]:
earnings = {'company': 'Apple',
 'ceo': 'Tim Cook',
 'cfo': 'Luca Maestri',
 'quarter': 'Q2 2024',
 'revenue': '$90.8 billion',
 'revenue_change': '-4%',
 'earnings_per_share': '$1.53',
 'dividend_per_share': '$0.25',
 'dividend_increase': '4%',
 'dividend_payable_date': 'May 16, 2024',
 'dividend_record_date': 'May 13, 2024',
 'share_repurchase_program': '$110 billion',
 'earnings_call_date': 'May 2, 2024',
 'earnings_call_time': '2:00 p.m. PT',
 'earnings_call_webcast': 'apple.com/investor/earnings-call'}

js = json.dumps(earnings)

In [34]:
prompt = f"""
Please convert the following JSON document enclosed in triple backticks into a quarterly earnings report suitable for shareholders. \n\n
```
{js}
```
"""

completion = complete(prompt)
print_response(completion)

**Apple Inc. Quarterly Earnings Report - Q2 2024**

- **Company:** Apple Inc.
- **CEO:** Tim Cook
- **CFO:** Luca Maestri
- **Quarter:** Q2 2024

**Financial Highlights:**
- **Revenue:** $90.8 billion
- **Revenue Change:** -4%
- **Earnings Per Share:** $1.53

**Dividend Information:**
- **Dividend Per Share:** $0.25
- **Dividend Increase:** 4%
- **Dividend Payable Date:** May 16, 2024
- **Dividend Record Date:** May 13, 2024

**Shareholder Programs:**
- **Share Repurchase Program:** $110 billion

**Earnings Call Details:**
- **Earnings Call Date:** May 2, 2024
- **Earnings Call Time:** 2:00 p.m. PT
- **Earnings Call Webcast:** [Apple Investor Earnings Call](apple.com/investor/earnings-call)

For more detailed financial information, please refer to the official quarterly earnings report released by Apple Inc.

### JSON Mode

https://platform.openai.com/docs/guides/text-generation/json-mode

In [35]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON"},
        {"role": "user", "content": "Who won the world series in 2020?"}
    ]
)
print(completion.choices[0].message.content)

{
  "winner": "Los Angeles Dodgers",
  "year": 2020,
  "final_opponent": "Tampa Bay Rays"
}


### Translation

In [37]:
def complete(user_prompt):
    completion = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return completion

def print_response(completion):
    content = completion.choices[0].message.content
    print(content)

In [38]:
prompt = f"""
Translate the following English text to Spanish: \
```
Hello, how are you?
```
"""

completion = complete(prompt)
print_response(completion)

Hola, ¿cómo estás?


In [39]:
prompt = f"""
Translate the following English text to nerdy scientist: \
```
I'm hungry. Can I have McNuggets?
```
"""

completion = complete(prompt)
print_response(completion)

My caloric intake is depleted. May I procure some McNuggets?


In [41]:
# Translating arbitary languages
# Use-case: building a customer service bot
# You don't know what language customers will write in!
# But can't let the bot reply in any language?
# Sure, but what if your company documents are in English?

user_message = "வணக்கம் எப்படி இருக்கிறாய்?"
prompt = f"""
What language is this message in? \
State only the language.
```
{user_message}
```
"""
completion = complete(prompt)
print_response(completion)

Tamil


In [43]:
# What language will it reply in?
prompt = f"""
Please write a polite customer service response to a customer who has sent the following message, reply in same language:
```
{user_message}
```
"""

completion = complete(prompt)
print_response(completion)

வணக்கம்,

நான் நன்றாக இருக்கிறேன், நீங்கள் எப்படி உதவ முடியும்? உங்கள் கேட்கைக்கு உதவ மிக்க மகிழ்ச்சி அடைகிறேன்.

உதவி தேவையானபடி என்னை அறியவும்.

நன்றி,
[உங்கள் பெயர்]


### Tone Enhancement

In [44]:
reply = "Why didn't you meet the prerequisites for this course you bozo?"
prompt = f"""
Please rewrite the following customer service reply to be extremely polite and compassionate towards the customer/student:
```
{reply}
```
"""

completion = complete(prompt)
print_response(completion)

We apologize for any confusion or misunderstanding that may have occurred regarding the prerequisites for this course. It appears that there may have been a mistake or oversight in the registration process. We would like to offer our assistance in helping you meet the necessary requirements to enroll in the course. Please let us know how we can support you in this matter. Thank you for bringing this to our attention.


### Sentiment Analysis

In [47]:
system_prompt = """
You are a helpful assistant with expert-level emotional intelligence. \
You reply only with 1, 0 or -1, with 1 meaning positive sentiment, -1 meaning negative sentiment, and 0 meaning neutral.
"""

def complete(user_prompt):
    completion = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return completion

def get_sentiment(text):
    user_prompt = f"""
    Please assign a sentiment value for the following text:\
    ```
    {text}
    ```
    """
    completion = complete(user_prompt)
    return completion.choices[0].message.content

In [48]:
review = """
Too melodramatic and not enough action. \
I watched this with my kid and he lost interest. \
I put on the old one instead and he liked it. \
Theres too much talking in this new one to be kid friendly. \
They go to another scene and they talk combine with separate x-men then they talk some more. \
Its suppose to be a cartoon tv show made for kids right? \
The original X-Men dialogue is to the point that scenes fly. \
This one a scene stays on the screen you would think this is an anime. \
And thats also my problem. It no longer feels like an americanized cartoon. \
If you have watched Teenage Mutant Ninja Turtles show than you know what I mean. \
Theres a certain pacing and forced emotion that alot of anime is known for.
"""
get_sentiment(review)

'-1'

### Stock Trading

In [49]:
system_prompt = """
You are a helpful assistant and an expert-level stock trader. \
You reply only with 1, 0 or -1, with 1 meaning 'good investment', -1 meaning 'poor investment', and 0 meaning neutral.
"""

def complete(user_prompt):
    completion = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
    )
    return completion

def get_trade(text):
    user_prompt = f"""
    Please tell me whether this is a good investment, poor investment, or neither, based on the following article about the company:\
    ```
    {text}
    ```
    """
    completion = complete(user_prompt)
    return completion.choices[0].message.content

In [51]:
article = "Consumer electronics giant Apple (AAPL) is looking to stand apart from other tech firms working on artificial intelligence by focusing on user privacy."
get_trade(article)

'1'

### Summarization

In [54]:
def complete(user_prompt):
    completion = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
        max_tokens=200,
    )
    return completion

def print_response(completion):
    content = completion.choices[0].message.content
    print(content)

In [53]:
stem_cells = f"""
What are stem cells, and why are they important?
Stem cells have the remarkable potential to renew themselves. They can develop into many different cell types in the body during early life and growth. Researchers study many different types of stem cells. There are several main categories: the “pluripotent” stem cells (embryonic stem cells and induced pluripotent stem cells) and nonembryonic or somatic stem cells (commonly called “adult” stem cells). Pluripotent stem cells have the ability to differentiate into all of the cells of the adult body. Adult stem cells are found in a tissue or organ and can differentiate to yield the specialized cell types of that tissue or organ.

Pluripotent stem cells

Early mammalian embryos at the blastocyst stage contain two types of cells – cells of the inner cell mass, and cells of the trophectoderm. The trophectodermal cells contribute to the placenta. The inner cell mass will ultimately develop into the specialized cell types, tissues, and organs of the entire body of the organism. Previous work with mouse embryos led to the development of a method in 1998 to derive stem cells from the inner cell mass of preimplantation human embryos and to grow human embryonic stem cells (hESCs) in the laboratory. In 2006, researchers identified conditions that would allow some mature human adult cells to be reprogrammed into an embryonic stem cell-like state. Those reprogramed stem cells are called induced pluripotent stem cells (iPSCs).

Adult stem cells

Throughout the life of the organism, populations of adult stem cells serve as an internal repair system that generates replacements for cells that are lost through normal wear and tear, injury, or disease. Adult stem cells have been identified in many organs and tissues and are generally associated with specific anatomical locations. These stem cells may remain quiescent (non-dividing) for long periods of time until they are activated by a normal need for more cells to maintain and repair tissues.
"""

In [56]:
prompt = f"""
Please summarize the following article. \
Use a maximum of 3 sentences:
```
{stem_cells}
```
"""

completion = complete(prompt)
print_response(completion)

Stem cells are important because they have the potential to renew themselves and develop into different cell types in the body. There are two main categories of stem cells: pluripotent stem cells, which can differentiate into all cell types, and adult stem cells, which are found in specific tissues and organs for repair and maintenance. Pluripotent stem cells can be derived from early mammalian embryos or reprogrammed from mature adult cells, while adult stem cells serve as an internal repair system throughout an organism's life.


### ELI5 (Explain like I'm 5)

In [58]:
prompt = f"""
Please ELI5 (explain like I'm 5) the following article.
```
{stem_cells}
```
"""

completion = complete(prompt)
print_response(completion)

Stem cells are special cells in our bodies that can turn into different types of cells. There are two main types of stem cells: pluripotent stem cells and adult stem cells. Pluripotent stem cells can turn into any type of cell in the body, while adult stem cells can only turn into specific types of cells in the organ or tissue they are found in.

Pluripotent stem cells come from early embryos and can be grown in a lab to study and potentially use for medical treatments. Adult stem cells are found in our bodies throughout our lives and help repair and replace damaged cells.

Stem cells are important because they have the potential to help treat diseases and injuries by replacing damaged cells with healthy ones. Researchers study stem cells to learn more about how they work and how they can be used to improve our health.


In [59]:
prompt = "Please ELI5 (explain like I'm 5) how airplanes work."
completion = complete(prompt)
print_response(completion)

Airplanes work by using the principles of lift, thrust, drag, and weight. 

1. Lift: Airplanes have wings that are shaped in a way that creates lift when air flows over them. This lift helps the airplane stay in the air.

2. Thrust: Airplanes have engines that provide thrust, which is the force that moves the airplane forward. This thrust helps the airplane move through the air.

3. Drag: Drag is the force that resists the motion of the airplane through the air. Airplanes are designed to minimize drag to help them fly more efficiently.

4. Weight: Weight is the force of gravity pulling the airplane down towards the ground. The lift generated by the wings must be greater than the weight of the airplane in order for it to stay in the air.

By balancing these forces and using control surfaces like ailerons, elevators, and rudders, pilots are able to control the airplane's direction, altitude, and speed.


### Question Answering

In [63]:
system_prompt = """
You are a helpful assistant that answers questions truthfully and factually. \
Your answer should be short and concise. \
If you don't know the answer, say so.
"""

def complete(user_prompt):
    completion = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
        max_tokens=100
    )
    return completion

def answer(question, context):
    prompt = f"""
    Please answer the following question:

    Question:
    ```
    {question}
    ```
    Use the following context to find the answer:
    ```
    {context}
    ```
    """

    completion = complete(prompt)
    content = completion.choices[0].message.content
    print(content)

In [61]:
bio = """
Dr. Jonathan Whitmore was a pioneering physicist born in 1803 in the quaint village of Ashbourne, England. From an early age, Whitmore exhibited an insatiable curiosity about the natural world, often conducting rudimentary experiments in his family's barn. He pursued formal education at the University of Cambridge, where he earned his doctorate in physics in 1827.

Whitmore's groundbreaking work in electromagnetism earned him widespread acclaim. In 1835, he published his seminal paper, "On the Interactions of Electric Currents and Magnetic Fields," which laid the foundation for future developments in electromagnetic theory. His meticulous experiments demonstrated the principles of electromagnetic induction, predating Faraday's similar discoveries.

In addition to his scientific contributions, Whitmore was an avid educator, holding a professorship at the Royal Institution of Great Britain. He mentored numerous students who would go on to make significant advancements in the field of physics.

Dr. Whitmore passed away in 1872, leaving behind a legacy of scientific inquiry and discovery that continues to inspire physicists to this day.
"""

In [64]:
q = "When was Dr. Jonathan Whitmore born?"
answer(q, bio)

Dr. Jonathan Whitmore was born in 1803.


In [65]:
q = "How tall was Dr. Jonathan Whitmore?"
answer(q, bio)

I'm sorry, but the provided context does not mention Dr. Jonathan Whitmore's height.


In [66]:
q = "What was Dr. Jonathan Whitmore field of study?"
answer(q, bio)

Dr. Jonathan Whitmore's field of study was physics.


In [67]:
q = "What later discoveries did Dr. Jonathan Whitmore inspire?"
answer(q, bio)

I'm sorry, but there is no mention of Dr. Jonathan Whitmore inspiring any specific later discoveries in the provided context.


### Chain of Thought

Breaking a problem into steps and makes reasoning explicit and increase the chance of arriving at the correct answer.

In [69]:
def complete(user_prompt):
    completion = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role": "user", "content": user_prompt}
        ],
        temperature=0,
        max_tokens=100
    )
    return completion

def print_response(completion):
    content = completion.choices[0].message.content
    print(content)

In [73]:
prompt = """
I went to the market and bought 10 apples. \
I gave 2 apples to the neighbor and 2 to the repairman. \
I then bought 5 more apples and ate 1. \
How many apples do I have now?
"""

completion = complete(prompt)
print_response(completion)

After giving away 4 apples and eating 1, I have 10 - 4 - 1 = 5 apples left.


In [74]:
prompt = """
A juggler can juggle 16 balls. \
Half of the balls are golf balls, and half of the golf balls are blue. \
How many blue golf balls are there?
"""

completion = complete(prompt)
print_response(completion)

There are 4 blue golf balls. 

Since half of the balls are golf balls, there are 8 golf balls in total. And since half of the golf balls are blue, there are 4 blue golf balls.


In [75]:
prompt = """
A juggler can juggle 16 balls. \
Half of the balls are golf balls, and half of the golf balls are blue. \
How many blue golf balls are there?

Let's think step by step.
"""

completion = complete(prompt)
print_response(completion)

1. The juggler can juggle 16 balls in total.
2. Half of the balls are golf balls, so there are 16/2 = 8 golf balls.
3. Half of the golf balls are blue, so there are 8/2 = 4 blue golf balls.

Therefore, there are 4 blue golf balls.
